# DEMO of response format capabilities
As you can see in the below made up example, the LLM is FORCED to produce outputs in the format laid out in the class called Test. It's important that Test inherits from Pydantic's BaseModel, as this is handled internally by the openai SDK and is also used to parse the text json output back into the class. Once you have the output object, you can access the attributes of the object like normal. The use of Typing is enforced by openai, ensuring that a string is always a string, an integer is always an integer, a List is always a list, and a Literal has to be one of the options you give it. This is perfect for ensuring LLM outputs are bulletproof in their conforming to your expected outputs, meaning that they do not break unexpectedly no matter what the input is.

The use of Field, from pydantic allows you to add a description property to the attribute. This description is provided to the LLM too, and so are any docstrings!

In [6]:
from pfd_toolkit.llm import LLM
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Literal
load_dotenv()
class Test(BaseModel):
    """An example response format case, this text is seen by the LLM too!"""
    name: str = Field(..., description='The name of the patient')
    sibling_ages: List[str] = Field(..., description='The ages of their siblings')
    death_reason: str = Field(..., description='A verbose reason for the patients death')
    category: Literal['suicide', 'old-age', 'boreddd'] = Field(..., description='Which category the death reason falls into')

In [7]:
llm = LLM(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-4o-mini')

In [8]:
res = llm.generate(prompt='hi!', response_format=Test)

In [9]:
res

Test(name='John Doe', sibling_ages=['25', '30', '22'], death_reason='John Doe passed away due to complications arising from a long-term battle with depression, which ultimately led to his decision to take his own life.', category='suicide')

In [10]:
print(res.name)
print(res.death_reason)
print(res.sibling_ages)
print(res.category)

John Doe
John Doe passed away due to complications arising from a long-term battle with depression, which ultimately led to his decision to take his own life.
['25', '30', '22']
suicide
